In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [6]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []

for w in words[:5]:
  print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '---->', itos[ix])
    context = context[1:] + [ix] # rolling window

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ----> e
..e ----> m
.em ----> m
emm ----> a
mma ----> .
olivia
... ----> o
..o ----> l
.ol ----> i
oli ----> v
liv ----> i
ivi ----> a
via ----> .
ava
... ----> a
..a ----> v
.av ----> a
ava ----> .
isabella
... ----> i
..i ----> s
.is ----> a
isa ----> b
sab ----> e
abe ----> l
bel ----> l
ell ----> a
lla ----> .
sophia
... ----> s
..s ----> o
.so ----> p
sop ----> h
oph ----> i
phi ----> a
hia ----> .


In [7]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [8]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])

In [9]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [10]:
# embeddings table
# take vectors and embed them in lower dimensional space
# we have 27 possible characters, we put them in 2-dimensional space

C = torch.randn((27, 2)) # embeddings table

In [11]:
C[5] # this embeds an integer into 2 dimensional vector 

tensor([0.4408, 0.5869])

In [12]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C # this embeds an integer into 2 dimensional vector 

tensor([0.4408, 0.5869])

In [13]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [14]:
# neural network first layer (hidden layer)
W1 = torch.randn((6, 100)) # weights
b1 = torch.randn(100) # biases

In [36]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # hidden layer with activation function
h

tensor([[-0.8038,  0.3956, -0.9997,  ..., -0.0458,  0.9923, -0.9845],
        [-0.1444,  0.9996, -0.9732,  ..., -0.9134,  0.9998,  0.5646],
        [-0.9993, -0.9591, -0.7936,  ...,  0.9972,  0.9997, -0.9999],
        ...,
        [-0.3809, -0.8856, -0.8136,  ...,  0.4825,  0.3297,  0.7170],
        [-0.8818,  0.6640,  0.9513,  ..., -0.4964,  0.8908,  0.9823],
        [-0.9994,  0.5703,  0.7077,  ...,  0.9947,  0.9999,  0.2868]])

In [37]:
# neural network second layer (output layer)
W2 = torch.randn((100, 27)) # weights
b2 = torch.randn(27) # biases

In [38]:
logits = h @ W2 + b2

In [39]:
logits.shape

torch.Size([32, 27])

In [40]:
counts = logits.exp()

In [42]:
prob = counts / counts.sum(1, keepdim=True)

In [43]:
prob.shape

torch.Size([32, 27])

In [47]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(16.2622)

In [ ]:
# NOW REFACTORED MLP

In [48]:
# dataset
X.shape, Y.shape

(torch.Size([32, 3]), torch.Size([32]))

In [49]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator=g) # embeddings table
# neural network first layer (hidden layer)
W1 = torch.randn((6, 100), generator=g) # weights
b1 = torch.randn(100, generator=g) # biases
# neural network second layer (output layer)
W2 = torch.randn((100, 27), generator=g) # weights
b2 = torch.randn(27, generator=g) # biases
parameters = [C, W1, b1, W2, b2]

In [50]:
sum(p.nelement() for p in parameters) # number of parameters in total

3481

In [52]:
emb = C[X] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # hidden layer with activation function, (32, 100)
logits = h @ W2 + b2
counts = logits.exp()
prob = counts / counts.sum(1, keepdim=True)
loss = -prob[torch.arange(32), Y].log().mean()
loss


tensor(17.7697)